# UniModal Images Performance

In [3]:
import os

def get_folder_size(folder):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            
            if not os.path.islink(file_path):
                total_size += os.path.getsize(file_path)
    return total_size

folder = "flattened_images_2"
size_bytes = get_folder_size(folder)
size_mb = size_bytes / (1024 * 1024)
size_gb = size_bytes / (1024 * 1024 * 1024)

print(f"Total size of '{folder}':")
print(f"  {size_bytes} bytes")
print(f"  {size_mb:.2f} MB")
print(f"  {size_gb:.2f} GB")


Total size of 'flattened_images_2':
  84489294400 bytes
  80575.27 MB
  78.69 GB


In [2]:
!nvidia-smi

Tue Apr 15 14:13:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.04             Driver Version: 570.124.04     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A4000               Off |   00000000:01:00.0 Off |                  Off |
| 41%   41C    P8             16W /  140W |     431MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import AutoImageProcessor, DeiTForImageClassification
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import classification_report

#  Check CUDA availability
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths to datasets and preprocessed images
train_csv = "train_data_3.csv"
val_csv = "val_data_3.csv"
test_csv = "test_data_3.csv"
image_dir = "preprocessed_images_3"  

# Load Data Splits
train_data = pd.read_csv(train_csv)
val_data = pd.read_csv(val_csv)
test_data = pd.read_csv(test_csv)

#  Load DeiT-Base Model and Image Processor with classifier head
checkpoint = "facebook/deit-base-distilled-patch16-224"  
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = DeiTForImageClassification.from_pretrained(
    checkpoint, num_labels=3, ignore_mismatched_sizes=True  
).to(device)

# Define label mapping 
label_map = {"CN": 0, "MCI": 1, "AD": 2}

class MRIImageDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        
        img_path = os.path.join(self.image_dir, f"{row['Image Data ID']}.png")
        
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = label_map[row["Group"]]
        return image, label

#  Define Transformations for DeiT-Base
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

#  Create PyTorch Dataloaders
batch_size = 8
train_dataset = MRIImageDataset(train_data, image_dir, transform=transform)
val_dataset = MRIImageDataset(val_data, image_dir, transform=transform)
test_dataset = MRIImageDataset(test_data, image_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

#  Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

#  Training Loop with Early Stopping
epochs = 10
best_val_loss = float("inf")
best_model_state = None
patience = 3
wait = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_dataset)
    train_acc = correct / total

    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            correct_val += predicted.eq(labels).sum().item()
            total_val += labels.size(0)

    val_loss = running_val_loss / len(val_dataset)
    val_acc = correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        wait = 0
        print("Best model saved.")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

if best_model_state:
    model.load_state_dict(best_model_state)

#  Evaluate Model on Test Set and Compute Classification Report
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = correct / total
print(f"🔥 Test Accuracy: {test_acc:.4f}")

# Compute and print classification report 
report = classification_report(all_labels, all_preds, target_names=list(label_map.keys()))
print("Classification Report:")
print(report)


Using device: cuda:1


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


Epoch 1/10 | Train Loss: 1.0638, Train Acc: 0.4623 | Val Loss: 1.0175, Val Acc: 0.4971
Best model saved.
Epoch 2/10 | Train Loss: 0.9520, Train Acc: 0.5439 | Val Loss: 0.8992, Val Acc: 0.6192
Best model saved.
Epoch 3/10 | Train Loss: 0.4299, Train Acc: 0.8243 | Val Loss: 1.0173, Val Acc: 0.5610
Epoch 4/10 | Train Loss: 0.0396, Train Acc: 0.9938 | Val Loss: 1.1181, Val Acc: 0.6337
Epoch 5/10 | Train Loss: 0.0086, Train Acc: 0.9988 | Val Loss: 1.1097, Val Acc: 0.6773
Early stopping triggered.
🔥 Test Accuracy: 0.6522
Classification Report:
              precision    recall  f1-score   support

          CN       0.61      0.59      0.60       106
         MCI       0.69      0.76      0.72       167
          AD       0.62      0.49      0.55        72

    accuracy                           0.65       345
   macro avg       0.64      0.61      0.62       345
weighted avg       0.65      0.65      0.65       345



In [8]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import AutoImageProcessor, DeiTForImageClassification
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import classification_report

#  Check CUDA availability
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths to datasets and preprocessed PNG images
train_csv = "train_data_3.csv"
val_csv = "val_data_3.csv"
test_csv = "test_data_3.csv"
image_dir = "preprocessed_images_3"  

# Load Data Splits
train_data = pd.read_csv(train_csv)
val_data = pd.read_csv(val_csv)
test_data = pd.read_csv(test_csv)

#  Load DeiT-Base Model and Image Processor with classifier head
checkpoint = "facebook/deit-base-distilled-patch16-224"  
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = DeiTForImageClassification.from_pretrained(
    checkpoint, num_labels=3, ignore_mismatched_sizes=True  
).to(device)

# Define label mapping 
label_map = {"CN": 0, "MCI": 1, "AD": 2}

class MRIImageDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        # Build the file path assuming PNG images
        img_path = os.path.join(self.image_dir, f"{row['Image Data ID']}.png")
        # Open the PNG image using PIL and convert it to RGB
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = label_map[row["Group"]]
        return image, label

#  Define Transformations for DeiT-Base
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

#  Create PyTorch Dataloaders
batch_size = 8
train_dataset = MRIImageDataset(train_data, image_dir, transform=transform)
val_dataset = MRIImageDataset(val_data, image_dir, transform=transform)
test_dataset = MRIImageDataset(test_data, image_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

#  Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

#  Training Loop with Early Stopping
epochs = 10
best_val_loss = float("inf")
best_model_state = None
patience = 3
wait = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_dataset)
    train_acc = correct / total

    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            correct_val += predicted.eq(labels).sum().item()
            total_val += labels.size(0)

    val_loss = running_val_loss / len(val_dataset)
    val_acc = correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        wait = 0
        print("Best model saved.")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

if best_model_state:
    model.load_state_dict(best_model_state)

#  Evaluate Model on Test Set and Compute Classification Report
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = correct / total
print(f"🔥 Test Accuracy: {test_acc:.4f}")

# Compute and print classification report using scikit-learn
report = classification_report(all_labels, all_preds, target_names=list(label_map.keys()))
print("Classification Report:")
print(report)


2025-03-19 05:59:31.817615: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-19 05:59:31.856938: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-19 05:59:32.474935: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using device: cuda:1


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10 | Train Loss: 1.0688, Train Acc: 0.4579 | Val Loss: 1.0299, Val Acc: 0.4855
Best model saved.
Epoch 2/10 | Train Loss: 0.9702, Train Acc: 0.5445 | Val Loss: 0.9788, Val Acc: 0.5174
Best model saved.
Epoch 3/10 | Train Loss: 0.4673, Train Acc: 0.8187 | Val Loss: 0.8478, Val Acc: 0.6570
Best model saved.
Epoch 4/10 | Train Loss: 0.0413, Train Acc: 0.9963 | Val Loss: 0.9522, Val Acc: 0.6453
Epoch 5/10 | Train Loss: 0.0045, Train Acc: 1.0000 | Val Loss: 1.0168, Val Acc: 0.6599
Epoch 6/10 | Train Loss: 0.0017, Train Acc: 1.0000 | Val Loss: 1.0938, Val Acc: 0.6628
Early stopping triggered.
🔥 Test Accuracy: 0.6435
Classification Report:
              precision    recall  f1-score   support

          CN       0.62      0.52      0.56       106
         MCI       0.66      0.80      0.72       167
          AD       0.63      0.46      0.53        72

    accuracy                           0.64       345
   macro avg       0.64      0.59      0.61       345
weighted avg       0.64  

## DEIT Model End to end training and evaluation

In [12]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import AutoImageProcessor, DeiTForImageClassification
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Check CUDA availability
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths to datasets and preprocessed images
train_csv = "train_data_3.csv"
val_csv = "val_data_3.csv"
test_csv = "test_data_3.csv"
image_dir = "preprocessed_images_3"

# Load Data Splits
train_data = pd.read_csv(train_csv)
val_data = pd.read_csv(val_csv)
test_data = pd.read_csv(test_csv)

# Load DeiT-Base Model and Image Processor with classifier head
checkpoint = "facebook/deit-base-distilled-patch16-224"  
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = DeiTForImageClassification.from_pretrained(
    checkpoint, num_labels=3, ignore_mismatched_sizes=True  
).to(device)

# Define label mapping 
label_map = {"CN": 0, "MCI": 1, "AD": 2}

class MRIImageDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        img_path = os.path.join(self.image_dir, f"{row['Image Data ID']}.png")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = label_map[row["Group"]]
        return image, label

# Define Transformations for DeiT-Base
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

# Create PyTorch Dataloaders
batch_size = 8
train_dataset = MRIImageDataset(train_data, image_dir, transform=transform)
val_dataset = MRIImageDataset(val_data, image_dir, transform=transform)
test_dataset = MRIImageDataset(test_data, image_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Training Loop with Early Stopping
epochs = 10
best_val_loss = float("inf")
best_model_state = None
patience = 3
wait = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_dataset)
    train_acc = correct / total

    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            correct_val += predicted.eq(labels).sum().item()
            total_val += labels.size(0)

    val_loss = running_val_loss / len(val_dataset)
    val_acc = correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        wait = 0
        print("Best model saved.")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

if best_model_state:
    model.load_state_dict(best_model_state)

# Evaluate Model on Test Set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"🔥 Test Accuracy: {test_acc:.4f}")


2025-03-17 05:45:45.859845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 05:45:45.897912: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-17 05:45:46.446734: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using device: cuda:1


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10 | Train Loss: 1.0628, Train Acc: 0.4654 | Val Loss: 1.0162, Val Acc: 0.5174
Best model saved.
Epoch 2/10 | Train Loss: 0.9562, Train Acc: 0.5340 | Val Loss: 0.9823, Val Acc: 0.5291
Best model saved.
Epoch 3/10 | Train Loss: 0.5160, Train Acc: 0.8062 | Val Loss: 0.8068, Val Acc: 0.6512
Best model saved.
Epoch 4/10 | Train Loss: 0.0677, Train Acc: 0.9931 | Val Loss: 0.9239, Val Acc: 0.6657
Epoch 5/10 | Train Loss: 0.0105, Train Acc: 1.0000 | Val Loss: 0.9995, Val Acc: 0.6628
Epoch 6/10 | Train Loss: 0.0044, Train Acc: 1.0000 | Val Loss: 1.0697, Val Acc: 0.6541
Early stopping triggered.
🔥 Test Accuracy: 0.6348


## Margin of Errors for deit

In [4]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import AutoImageProcessor, DeiTForImageClassification
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#  Check CUDA availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths to datasets and preprocessed images
train_csv = "train_data_3.csv"
val_csv = "val_data_3.csv"
test_csv = "test_data_3.csv"
image_dir = "preprocessed_images_3"

# Load Data Splits
train_data = pd.read_csv(train_csv)
val_data = pd.read_csv(val_csv)
test_data = pd.read_csv(test_csv)

#  Load DeiT-Base Model and Image Processor with classifier head
checkpoint = "facebook/deit-base-distilled-patch16-224"  # Using DeiT base distilled model
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = DeiTForImageClassification.from_pretrained(
    checkpoint, num_labels=3, ignore_mismatched_sizes=True  
).to(device)

# Define label mapping (example: "CN": Cognitively Normal, "MCI": Mild Cognitive Impairment, "AD": Alzheimer’s Disease)
label_map = {"CN": 0, "MCI": 1, "AD": 2}

class MRIImageDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        img_path = os.path.join(self.image_dir, f"{row['Image Data ID']}.png")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = label_map[row["Group"]]
        return image, label

#  Define Transformations for DeiT-Base
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

#  Create PyTorch Dataloaders
batch_size = 8
train_dataset = MRIImageDataset(train_data, image_dir, transform=transform)
val_dataset = MRIImageDataset(val_data, image_dir, transform=transform)
test_dataset = MRIImageDataset(test_data, image_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

#  Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

#  Training Loop with Early Stopping
epochs = 10
best_val_loss = float("inf")
best_model_state = None
patience = 3
wait = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_dataset)
    train_acc = correct / total

    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            correct_val += predicted.eq(labels).sum().item()
            total_val += labels.size(0)

    val_loss = running_val_loss / len(val_dataset)
    val_acc = correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        wait = 0
        print("Best model saved.")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

if best_model_state:
    model.load_state_dict(best_model_state)

#  Evaluate Model on Test Set
model.eval()
correct = 0
total = 0
all_preds = []
all_targets = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

test_acc = correct / total
print(f"🔥 Test Accuracy: {test_acc:.4f}")



# Approach 1: Direct Calculation using the Binomial Formula.
n_test = len(test_dataset)
se_direct = math.sqrt(test_acc * (1 - test_acc) / n_test)
moe_direct = 1.96 * se_direct
print("\nDirect Calculation:")
print("Standard Error: {:.4f}".format(se_direct))
print("Margin of Error (95% CI): {:.4f}".format(moe_direct))
print("95% Confidence Interval for Accuracy: [{:.4f}, {:.4f}]".format(test_acc - moe_direct, test_acc + moe_direct))

# Approach 2: Bootstrapping.
n_bootstrap = 1000
boot_accs = []
all_preds = np.array(all_preds)
all_targets = np.array(all_targets)
for i in range(n_bootstrap):
    indices = np.random.choice(np.arange(n_test), n_test, replace=True)
    acc_bs = accuracy_score(all_targets[indices], all_preds[indices])
    boot_accs.append(acc_bs)
boot_accs = np.array(boot_accs)
se_bootstrap = np.std(boot_accs)
moe_bootstrap = 1.96 * se_bootstrap
print("\nBootstrapping:")
print("Bootstrapped Mean Accuracy: {:.4f}".format(np.mean(boot_accs)))
print("Standard Error (Bootstrap): {:.4f}".format(se_bootstrap))
print("Margin of Error (Bootstrap, 95% CI): {:.4f}".format(moe_bootstrap))


Using device: cuda:0


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10 | Train Loss: 1.0613, Train Acc: 0.4698 | Val Loss: 1.0142, Val Acc: 0.4884
Best model saved.
Epoch 2/10 | Train Loss: 0.9641, Train Acc: 0.5240 | Val Loss: 0.9699, Val Acc: 0.5465
Best model saved.
Epoch 3/10 | Train Loss: 0.6085, Train Acc: 0.7639 | Val Loss: 0.9055, Val Acc: 0.6250
Best model saved.
Epoch 4/10 | Train Loss: 0.1029, Train Acc: 0.9863 | Val Loss: 0.9356, Val Acc: 0.6744
Epoch 5/10 | Train Loss: 0.0150, Train Acc: 1.0000 | Val Loss: 1.0320, Val Acc: 0.6657
Epoch 6/10 | Train Loss: 0.0048, Train Acc: 1.0000 | Val Loss: 1.1095, Val Acc: 0.6628
Early stopping triggered.
🔥 Test Accuracy: 0.6551

Direct Calculation:
Standard Error: 0.0256
Margin of Error (95% CI): 0.0502
95% Confidence Interval for Accuracy: [0.6049, 0.7052]

Bootstrapping:
Bootstrapped Mean Accuracy: 0.6551
Standard Error (Bootstrap): 0.0269
Margin of Error (Bootstrap, 95% CI): 0.0527


In [5]:
from sklearn.metrics import f1_score

# Compute the weighted F1 score on the test set.
weighted_f1 = f1_score(all_targets, all_preds, average='weighted')
print("Weighted F1 Score: {:.4f}".format(weighted_f1))

# Bootstrapping to estimate the margin of error for the weighted F1 score.
n_bootstrap = 1000
boot_f1s = []
n_samples = len(all_targets)
for i in range(n_bootstrap):
    indices = np.random.choice(np.arange(n_samples), n_samples, replace=True)
    f1_bs = f1_score(all_targets[indices], all_preds[indices], average='weighted')
    boot_f1s.append(f1_bs)
boot_f1s = np.array(boot_f1s)

std_f1 = np.std(boot_f1s)
moe_f1 = 1.96 * std_f1

print("Bootstrapped Weighted F1 Mean: {:.4f}".format(np.mean(boot_f1s)))
print("Standard Error (Bootstrap) for Weighted F1: {:.4f}".format(std_f1))
print("Margin of Error (Bootstrap, 95% CI) for Weighted F1: {:.4f}".format(moe_f1))


Weighted F1 Score: 0.6492
Bootstrapped Weighted F1 Mean: 0.6498
Standard Error (Bootstrap) for Weighted F1: 0.0269
Margin of Error (Bootstrap, 95% CI) for Weighted F1: 0.0528


In [5]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import AutoImageProcessor, DeiTForImageClassification
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import classification_report

# Check CUDA availability
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths to datasets and preprocessed PNG images
train_csv = "train_data_3.csv"
val_csv = "val_data_3.csv"
test_csv = "test_data_3.csv"
image_dir = "preprocessed_images_3"  

# Load Data Splits
train_data = pd.read_csv(train_csv)
val_data = pd.read_csv(val_csv)
test_data = pd.read_csv(test_csv)

# Load DeiT-Base Model and Image Processor with classifier head
checkpoint = "facebook/deit-base-distilled-patch16-224"  
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = DeiTForImageClassification.from_pretrained(
    checkpoint, num_labels=3, ignore_mismatched_sizes=True  
).to(device)

# Define label mapping 
label_map = {"CN": 0, "MCI": 1, "AD": 2}

class MRIImageDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        # Build the file path assuming PNG images
        img_path = os.path.join(self.image_dir, f"{row['Image Data ID']}.png")
        # Open the PNG image using PIL and convert it to RGB
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = label_map[row["Group"]]
        return image, label

# Define Transformations for DeiT-Base
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

# Create PyTorch Dataloaders
batch_size = 8
train_dataset = MRIImageDataset(train_data, image_dir, transform=transform)
val_dataset = MRIImageDataset(val_data, image_dir, transform=transform)
test_dataset = MRIImageDataset(test_data, image_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

#  Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

#  Training Loop with Early Stopping
epochs = 10
best_val_loss = float("inf")
best_model_state = None
patience = 3
wait = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_dataset)
    train_acc = correct / total

    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            correct_val += predicted.eq(labels).sum().item()
            total_val += labels.size(0)

    val_loss = running_val_loss / len(val_dataset)
    val_acc = correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        wait = 0
        print("Best model saved.")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

if best_model_state:
    model.load_state_dict(best_model_state)

# Evaluate Model on Test Set and Compute Classification Report
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = correct / total
print(f"🔥 Test Accuracy: {test_acc:.4f}")

# Compute and print classification report using scikit-learn
report = classification_report(all_labels, all_preds, target_names=list(label_map.keys()))
print("Classification Report:")
print(report)


Using device: cuda:1


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10 | Train Loss: 1.0635, Train Acc: 0.4741 | Val Loss: 1.0187, Val Acc: 0.4855
Best model saved.
Epoch 2/10 | Train Loss: 0.9260, Train Acc: 0.5657 | Val Loss: 0.9279, Val Acc: 0.5233
Best model saved.
Epoch 3/10 | Train Loss: 0.3414, Train Acc: 0.8729 | Val Loss: 0.9068, Val Acc: 0.5930
Best model saved.
Epoch 4/10 | Train Loss: 0.0400, Train Acc: 0.9907 | Val Loss: 0.9134, Val Acc: 0.6541
Epoch 5/10 | Train Loss: 0.0072, Train Acc: 0.9994 | Val Loss: 0.9495, Val Acc: 0.7122
Epoch 6/10 | Train Loss: 0.0014, Train Acc: 1.0000 | Val Loss: 1.0481, Val Acc: 0.6977
Early stopping triggered.
🔥 Test Accuracy: 0.6638
Classification Report:
              precision    recall  f1-score   support

          CN       0.70      0.57      0.62       106
         MCI       0.66      0.83      0.73       167
          AD       0.62      0.43      0.51        72

    accuracy                           0.66       345
   macro avg       0.66      0.61      0.62       345
weighted avg       0.66  

## Using MLP

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Set device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load DeiT features.
train_features = np.load("Deit_train_features_1.npy")  
val_features   = np.load("Deit_val_features_1.npy")      
test_features  = np.load("Deit_test_features_1.npy")     

print("Train features shape:", train_features.shape)
print("Val features shape:", val_features.shape)
print("Test features shape:", test_features.shape)


# Read the CSV files 
train_data = pd.read_csv("train_data_3.csv")
val_data   = pd.read_csv("val_data_3.csv")
test_data  = pd.read_csv("test_data_3.csv")


label_col = "Group"

# Encode labels using LabelEncoder.
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data[label_col])
val_labels   = label_encoder.transform(val_data[label_col])
test_labels  = label_encoder.transform(test_data[label_col])

num_classes = len(label_encoder.classes_)


# Convert features and labels to tensors.
train_features = torch.tensor(train_features, dtype=torch.float32)
val_features   = torch.tensor(val_features, dtype=torch.float32)
test_features  = torch.tensor(test_features, dtype=torch.float32)

train_labels = torch.tensor(train_labels, dtype=torch.long)
val_labels   = torch.tensor(val_labels, dtype=torch.long)
test_labels  = torch.tensor(test_labels, dtype=torch.long)

# Create TensorDatasets.
train_dataset = TensorDataset(train_features, train_labels)
val_dataset   = TensorDataset(val_features, val_labels)
test_dataset  = TensorDataset(test_features, test_labels)

# Create DataLoaders.
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


class MLPClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLPClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),  
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),        
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes) 
        )
        
    def forward(self, x):
        return self.net(x)

# Example usage:
input_dim = train_features.shape[1]  
model_mlp = MLPClassifier(input_dim, num_classes)
model_mlp.to(device)



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mlp.parameters(), lr=2e-5)
num_epochs = 200
patience = 5  
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(num_epochs):
    model_mlp.train()
    train_loss = 0.0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model_mlp(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    train_loss /= len(train_dataset)
    
    # Evaluate on the validation set.
    model_mlp.eval()
    val_loss = 0.0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            logits = model_mlp(x)
            loss = criterion(logits, y)
            val_loss += loss.item() * x.size(0)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds.cpu().numpy())
            all_targets.append(y.cpu().numpy())
    val_loss /= len(val_dataset)
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    val_acc = accuracy_score(all_targets, all_preds)
    
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")
    
    # Early stopping check.
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model_mlp.state_dict()
        patience_counter = 0
        print("  -> New best model saved.")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

# Load the best model state.
model_mlp.load_state_dict(best_model_state)

##########################################
# 6. Evaluate on the Test Set
##########################################
model_mlp.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        logits = model_mlp(x)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.cpu().numpy())
        all_targets.append(y.cpu().numpy())
all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)
test_acc = accuracy_score(all_targets, all_preds)
print("Test Accuracy:", test_acc)
print("Classification Report:")
print(classification_report(all_targets, all_preds, target_names=[str(c) for c in label_encoder.classes_]))

Train features shape: (1605, 768)
Val features shape: (344, 768)
Test features shape: (345, 768)
Epoch 1/200: Train Loss = 1.0683, Val Loss = 1.0389, Val Acc = 0.4855
  -> New best model saved.
Epoch 2/200: Train Loss = 1.0490, Val Loss = 1.0353, Val Acc = 0.4855
  -> New best model saved.
Epoch 3/200: Train Loss = 1.0480, Val Loss = 1.0324, Val Acc = 0.4855
  -> New best model saved.
Epoch 4/200: Train Loss = 1.0396, Val Loss = 1.0306, Val Acc = 0.4855
  -> New best model saved.
Epoch 5/200: Train Loss = 1.0326, Val Loss = 1.0289, Val Acc = 0.4855
  -> New best model saved.
Epoch 6/200: Train Loss = 1.0371, Val Loss = 1.0265, Val Acc = 0.4855
  -> New best model saved.
Epoch 7/200: Train Loss = 1.0277, Val Loss = 1.0248, Val Acc = 0.4855
  -> New best model saved.
Epoch 8/200: Train Loss = 1.0270, Val Loss = 1.0230, Val Acc = 0.4855
  -> New best model saved.
Epoch 9/200: Train Loss = 1.0305, Val Loss = 1.0210, Val Acc = 0.4855
  -> New best model saved.
Epoch 10/200: Train Loss = 1.0

# Uni Modal Tabular Data performance

## Ft Transformer end to end training and evalaution

In [14]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  

import random
import numpy as np
import torch

seed = 42  # choose your seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)





In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from rtdl_revisiting_models import FTTransformer


# Load CSV files for train, validation, and test splits.
train_data = pd.read_csv("train_data_3.csv")
val_data   = pd.read_csv("val_data_3.csv")
test_data  = pd.read_csv("test_data_3.csv")

# Define feature columns and target label.
numerical_features = ["Age", "CDGLOBAL", "CDRSB", "MMSCORE", "HMSCORE", "NPISCORE", "GDTOTAL"]
categorical_features = ["GENOTYPE"]
label = "Group"

# Subset dataframes to desired columns.
cols = numerical_features + categorical_features + [label]
train_data = train_data[cols]
val_data   = val_data[cols]
test_data  = test_data[cols]

# Handle missingness for numerical features.
cols_with_missing = ["CDRSB", "MMSCORE", "HMSCORE", "NPISCORE", "GDTOTAL"]
for col in cols_with_missing:
    for df in [train_data, val_data, test_data]:
        df[col + "_is_missing"] = df[col].isnull().astype(int)
        df[col] = df[col].fillna(-999)

# Extend continuous features to include missing indicators.
numerical_features_extended = numerical_features + [col + "_is_missing" for col in cols_with_missing]

# Encode categorical features using LabelEncoder.
cat_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col].astype(str))
    val_data[col]   = le.transform(val_data[col].astype(str))
    test_data[col]  = le.transform(test_data[col].astype(str))
    cat_encoders[col] = le

# Encode the target.
label_encoder = LabelEncoder()
train_data[label] = label_encoder.fit_transform(train_data[label])
val_data[label]   = label_encoder.transform(val_data[label])
test_data[label]  = label_encoder.transform(test_data[label])
num_classes = len(label_encoder.classes_)  


# Continuous features.
X_train_cont = train_data[numerical_features_extended].values.astype(np.float32)
X_val_cont   = val_data[numerical_features_extended].values.astype(np.float32)
X_test_cont  = test_data[numerical_features_extended].values.astype(np.float32)

# Categorical features.
X_train_cat = train_data[categorical_features].values.astype(np.int64)
X_val_cat   = val_data[categorical_features].values.astype(np.int64)
X_test_cat  = test_data[categorical_features].values.astype(np.int64)

# Labels.
y_train = train_data[label].values.astype(np.int64)
y_val   = val_data[label].values.astype(np.int64)
y_test  = test_data[label].values.astype(np.int64)

# Create a simple PyTorch Dataset.
class TabularDataset(Dataset):
    def __init__(self, cont, cat, labels):
        self.cont = cont
        self.cat = cat
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            "cont": torch.tensor(self.cont[idx], dtype=torch.float32),
            "cat": torch.tensor(self.cat[idx], dtype=torch.long),
            "target": torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = TabularDataset(X_train_cont, X_train_cat, y_train)
val_dataset   = TabularDataset(X_val_cont, X_val_cat, y_val)
test_dataset  = TabularDataset(X_test_cont, X_test_cat, y_test)

# Create DataLoaders.
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Get the number of continuous features.
n_cont_features = X_train_cont.shape[1]
cat_cardinalities = [int(train_data[col].nunique()) for col in categorical_features]

# For classification, set d_out = number of classes.
d_out = num_classes

# Instantiate the FTTransformer.
model = FTTransformer(
    n_cont_features=n_cont_features,
    cat_cardinalities=cat_cardinalities,
    d_out=d_out,
    n_blocks=3,
    d_block=192,                
    attention_n_heads=8,
    attention_dropout=0.2,
    ffn_d_hidden=None,          
    ffn_d_hidden_multiplier=4/3,
    ffn_dropout=0.1,
    residual_dropout=0.0
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set up optimizer, loss function, and scheduler.
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# Training loop with early stopping.
max_epochs = 100
patience = 5
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(max_epochs):
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        cont = batch["cont"].to(device)
        cat = batch["cat"].to(device)
        targets = batch["target"].to(device)
        
        optimizer.zero_grad()
        logits = model(cont, cat) 
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * cont.size(0)
    train_loss /= len(train_dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            cont = batch["cont"].to(device)
            cat = batch["cat"].to(device)
            targets = batch["target"].to(device)
            
            logits = model(cont, cat)
            loss = criterion(logits, targets)
            val_loss += loss.item() * cont.size(0)
    val_loss /= len(val_dataset)
    
    scheduler.step(val_loss)
    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

# Save the best model.
save_model_path = "best_ft_transformer_classification.pt"
torch.save(best_model_state, save_model_path)
print("Trained model saved to", save_model_path)

# Load the best model.
model.load_state_dict(torch.load(save_model_path))

# Evaluate classification performance on the test set.
model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for batch in test_loader:
        cont = batch["cont"].to(device)
        cat = batch["cat"].to(device)
        targets = batch["target"].to(device)
        
        logits = model(cont, cat)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)
test_acc = accuracy_score(all_targets, all_preds)
print("Test Accuracy:", test_acc)
print("Classification Report (Test):")
print(classification_report(all_targets, all_preds, target_names=[str(c) for c in label_encoder.classes_]))

/home/sran-m36/Multi Modal Project/multimodal_env/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1: Train Loss: 1.0557 | Val Loss: 1.0274
Epoch 2: Train Loss: 1.0233 | Val Loss: 0.9525
Epoch 3: Train Loss: 0.7375 | Val Loss: 0.4822
Epoch 4: Train Loss: 0.4742 | Val Loss: 0.3990
Epoch 5: Train Loss: 0.4257 | Val Loss: 0.3785
Epoch 6: Train Loss: 0.3891 | Val Loss: 0.3898
Epoch 7: Train Loss: 0.3681 | Val Loss: 0.3462
Epoch 8: Train Loss: 0.3403 | Val Loss: 0.3471
Epoch 9: Train Loss: 0.3667 | Val Loss: 0.3960
Epoch 10: Train Loss: 0.3602 | Val Loss: 0.3369
Epoch 11: Train Loss: 0.3360 | Val Loss: 0.3430
Epoch 12: Train Loss: 0.3377 | Val Loss: 0.3402
Epoch 13: Train Loss: 0.3413 | Val Loss: 0.3201
Epoch 14: Train Loss: 0.3366 | Val Loss: 0.3385
Epoch 15: Train Loss: 0.3308 | Val Loss: 0.3481
Epoch 16: Train Loss: 0.3233 | Val Loss: 0.3290
Epoch 17: Train Loss: 0.3308 | Val Loss: 0.3324
Epoch 18: Train Loss: 0.3194 | Val Loss: 0.3325
Early stopping triggered.
Trained model saved to best_ft_transformer_classification.pt
Test Accuracy: 0.9043478260869565
Classification Report (T

/tmp/ipykernel_789196/921975057.py:191: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_model_path))


## Margin of errors for the ft-transformer model

In [19]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from rtdl_revisiting_models import FTTransformer  
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import math


# Load CSV files for train, validation, and test splits.
train_data = pd.read_csv("train_data_3.csv")
val_data   = pd.read_csv("val_data_3.csv")
test_data  = pd.read_csv("test_data_3.csv")

# Define feature columns and target label.
numerical_features = ["Age", "CDGLOBAL", "CDRSB", "MMSCORE", "HMSCORE", "NPISCORE", "GDTOTAL"]
categorical_features = ["GENOTYPE"]
label = "Group"

# Subset dataframes to desired columns.
cols = numerical_features + categorical_features + [label]
train_data = train_data[cols]
val_data   = val_data[cols]
test_data  = test_data[cols]

# Handle missingness for numerical features.
cols_with_missing = ["CDRSB", "MMSCORE", "HMSCORE", "NPISCORE", "GDTOTAL"]
for col in cols_with_missing:
    for df in [train_data, val_data, test_data]:
        # Create a missing indicator
        df[col + "_is_missing"] = df[col].isnull().astype(int)
        # Use a sentinel value (-999) for missing data
        df[col] = df[col].fillna(-999)

# Extend the list of continuous features to include missing indicators.
numerical_features_extended = numerical_features + [col + "_is_missing" for col in cols_with_missing]

# Encode categorical features.
cat_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col].astype(str))
    val_data[col]   = le.transform(val_data[col].astype(str))
    test_data[col]  = le.transform(test_data[col].astype(str))
    cat_encoders[col] = le

# Encode the target labels.
label_encoder = LabelEncoder()
train_data[label] = label_encoder.fit_transform(train_data[label])
val_data[label]   = label_encoder.transform(val_data[label])
test_data[label]  = label_encoder.transform(test_data[label])
num_classes = len(label_encoder.classes_)  


# Continuous features.
X_train_cont = train_data[numerical_features_extended].values.astype(np.float32)
X_val_cont   = val_data[numerical_features_extended].values.astype(np.float32)
X_test_cont  = test_data[numerical_features_extended].values.astype(np.float32)

# Categorical features.
X_train_cat = train_data[categorical_features].values.astype(np.int64)
X_val_cat   = val_data[categorical_features].values.astype(np.int64)
X_test_cat  = test_data[categorical_features].values.astype(np.int64)

# Labels.
y_train = train_data[label].values.astype(np.int64)
y_val   = val_data[label].values.astype(np.int64)
y_test  = test_data[label].values.astype(np.int64)

class TabularDataset(Dataset):
    def __init__(self, cont, cat, labels):
        self.cont = cont
        self.cat = cat
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            "cont": torch.tensor(self.cont[idx], dtype=torch.float32),
            "cat": torch.tensor(self.cat[idx], dtype=torch.long),
            "target": torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = TabularDataset(X_train_cont, X_train_cat, y_train)
val_dataset   = TabularDataset(X_val_cont, X_val_cat, y_val)
test_dataset  = TabularDataset(X_test_cont, X_test_cat, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

n_cont_features = X_train_cont.shape[1]
cat_cardinalities = [int(train_data[col].nunique()) for col in categorical_features]
d_out = num_classes  # Number of classes for classification

model = FTTransformer(
    n_cont_features=n_cont_features,
    cat_cardinalities=cat_cardinalities,
    d_out=d_out,
    n_blocks=3,
    d_block=192,                
    attention_n_heads=8,
    attention_dropout=0.2,
    ffn_d_hidden=None,
    ffn_d_hidden_multiplier=4/3,
    ffn_dropout=0.1,
    residual_dropout=0.0
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# 5. Set Up Optimizer, Loss, and Scheduler

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)


# 6. Training Loop with Early Stopping

max_epochs = 100
patience = 3
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(max_epochs):
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        cont = batch["cont"].to(device)
        cat = batch["cat"].to(device)
        targets = batch["target"].to(device)
        
        optimizer.zero_grad()
        logits = model(cont, cat)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * cont.size(0)
    train_loss /= len(train_dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            cont = batch["cont"].to(device)
            cat = batch["cat"].to(device)
            targets = batch["target"].to(device)
            
            logits = model(cont, cat)
            loss = criterion(logits, targets)
            val_loss += loss.item() * cont.size(0)
    val_loss /= len(val_dataset)
    
    scheduler.step(val_loss)
    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

# Save the best model.
save_model_path = "best_ft_transformer_classification.pt"
torch.save(best_model_state, save_model_path)
print("Trained model saved to", save_model_path)

# Load the best model.
model.load_state_dict(torch.load(save_model_path))


# 7. Evaluate on the Test Set

model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for batch in test_loader:
        cont = batch["cont"].to(device)
        cat = batch["cat"].to(device)
        targets = batch["target"].to(device)
        
        logits = model(cont, cat)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.cpu().numpy())
        all_targets.append(targets.cpu().numpy())
all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)
test_acc = accuracy_score(all_targets, all_preds)
print("Test Accuracy:", test_acc)
print("Classification Report (Test):")
print(classification_report(all_targets, all_preds, target_names=[str(c) for c in label_encoder.classes_]))

conf_matrix = confusion_matrix(all_targets, all_preds)
print("Confusion Matrix:")
print(conf_matrix)


# 8. Compute Margin of Error for Test Accuracy and Weighted F1 Score


# -- Accuracy Margin of Error --

# Approach 1: Direct Calculation (Binomial Formula)
n_test = len(test_dataset)
se_direct = math.sqrt(test_acc * (1 - test_acc) / n_test)
moe_direct = 1.96 * se_direct
print("\nDirect Calculation for Accuracy:")
print("Standard Error: {:.4f}".format(se_direct))
print("Margin of Error (95% CI): {:.4f}".format(moe_direct))
print("95% Confidence Interval for Accuracy: [{:.4f}, {:.4f}]".format(test_acc - moe_direct, test_acc + moe_direct))

# Approach 2: Bootstrapping for Accuracy
n_bootstrap = 1000
boot_accs = []
for i in range(n_bootstrap):
    indices = np.random.choice(np.arange(n_test), n_test, replace=True)
    acc_bs = accuracy_score(all_targets[indices], all_preds[indices])
    boot_accs.append(acc_bs)
boot_accs = np.array(boot_accs)
se_bootstrap = np.std(boot_accs)
moe_bootstrap = 1.96 * se_bootstrap
print("\nBootstrapping for Accuracy:")
print("Bootstrapped Mean Accuracy: {:.4f}".format(np.mean(boot_accs)))
print("Standard Error (Bootstrap): {:.4f}".format(se_bootstrap))
print("Margin of Error (Bootstrap, 95% CI): {:.4f}".format(moe_bootstrap))

# -- Weighted F1 Score Margin of Error --

from sklearn.metrics import f1_score
weighted_f1 = f1_score(all_targets, all_preds, average='weighted')
print("\nWeighted F1 Score:", weighted_f1)

# Bootstrapping for Weighted F1 Score
boot_f1s = []
for i in range(n_bootstrap):
    indices = np.random.choice(np.arange(n_test), n_test, replace=True)
    f1_bs = f1_score(all_targets[indices], all_preds[indices], average='weighted')
    boot_f1s.append(f1_bs)
boot_f1s = np.array(boot_f1s)
se_boot_f1 = np.std(boot_f1s)
moe_boot_f1 = 1.96 * se_boot_f1
print("\nBootstrapping for Weighted F1 Score:")
print("Bootstrapped Mean Weighted F1: {:.4f}".format(np.mean(boot_f1s)))
print("Standard Error (Bootstrap) for Weighted F1: {:.4f}".format(se_boot_f1))
print("Margin of Error (Bootstrap, 95% CI) for Weighted F1: {:.4f}".format(moe_boot_f1))



Epoch 1: Train Loss: 1.0941 | Val Loss: 1.0624
Epoch 2: Train Loss: 1.0491 | Val Loss: 1.0481
Epoch 3: Train Loss: 1.0447 | Val Loss: 1.0407
Epoch 4: Train Loss: 1.0392 | Val Loss: 1.0390
Epoch 5: Train Loss: 1.0342 | Val Loss: 1.0369
Epoch 6: Train Loss: 1.0341 | Val Loss: 1.0315
Epoch 7: Train Loss: 1.0240 | Val Loss: 1.0204
Epoch 8: Train Loss: 1.0204 | Val Loss: 1.0066
Epoch 9: Train Loss: 1.0051 | Val Loss: 0.9841
Epoch 10: Train Loss: 0.9869 | Val Loss: 0.9368
Epoch 11: Train Loss: 0.9405 | Val Loss: 0.8596
Epoch 12: Train Loss: 0.8710 | Val Loss: 0.7485
Epoch 13: Train Loss: 0.7629 | Val Loss: 0.6240
Epoch 14: Train Loss: 0.6741 | Val Loss: 0.5347
Epoch 15: Train Loss: 0.6215 | Val Loss: 0.4861
Epoch 16: Train Loss: 0.5702 | Val Loss: 0.4439
Epoch 17: Train Loss: 0.5395 | Val Loss: 0.4257
Epoch 18: Train Loss: 0.4969 | Val Loss: 0.4071
Epoch 19: Train Loss: 0.4943 | Val Loss: 0.3988
Epoch 20: Train Loss: 0.4719 | Val Loss: 0.3869
Epoch 21: Train Loss: 0.4724 | Val Loss: 0.3813
E

/tmp/ipykernel_160858/3551740628.py:192: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_model_path))



Bootstrapping for Accuracy:
Bootstrapped Mean Accuracy: 0.9036
Standard Error (Bootstrap): 0.0160
Margin of Error (Bootstrap, 95% CI): 0.0314

Weighted F1 Score: 0.9041073677435971

Bootstrapping for Weighted F1 Score:
Bootstrapped Mean Weighted F1: 0.9038
Standard Error (Bootstrap) for Weighted F1: 0.0158
Margin of Error (Bootstrap, 95% CI) for Weighted F1: 0.0311
